In [29]:
import pandas as pd

In [30]:
df = pd.read_csv("spotify_millsongdata.csv")

In [31]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [32]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [33]:
df.shape

(57650, 4)

In [34]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [35]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [36]:
df.head(10)

,artist,song,text
0,Queens Of The Stone Age,Era Vulgaris,I play a game 'til I'm dead \r\nOr on a magaz...
1,King Diamond,I'm Not A Stranger,I've been watching Lucy all week \r\nI've bee...
2,Slayer,Stain Of Mind,Imagine Humanity's decline \r\nStep inside my...
3,Gloria Estefan,Close My Eyes,If I could just close my eyes \r\nSee just wh...
4,Amy Grant,Takes A Little Time,[Chorus] \r\nIt takes a little time sometimes...
5,Hank Snow,City Of New Orleans,Ridin' on the City of New Orleans Illinois Cen...
6,Evanescence,Listen To The Rain,"[Rain] \r\nListen, listen \r\nListen, listen..."
7,Steve Miller Band,Heart Like A Wheel,I've got a heart like a wheel \r\nFeel like I...
8,Rod Stewart,Makin' Whoopee,"Another bride, another June \r\nAnother sunny..."
9,Vanilla Ice,Hooked,"1 2 3 4 Hit it, Hit it \r\n[Verse 1] \r\nYea..."


In [37]:
df['text'][0]

"I play a game 'til I'm dead  \r\nOr on a magazine  \r\nI wanna look like I'm dead  \r\nDancing on a string  \r\nI eat the food when it's there  \r\nMake a mess of things  \r\nGet that look everywhere  \r\nBut keep that look off my face  \r\n  \r\nEverybody else wanna fall in love  \r\nThere's no room for love in a modern sky  \r\nLiving in the era vulgaris  \r\nJust drool in the dark  \r\nAs you stare at the lights  \r\n  \r\nFrom a crumbling tower I see everything  \r\nLondon, Tokyo, New York City  \r\nThere's no love any place  \r\n  \r\nEverybody else wanna fall in love  \r\nThere's no room for love in a modern sky  \r\nLiving in the era vulgaris  \r\nJust drool in the dark  \r\nAs you stare at the lights  \r\n  \r\nAll we creatures helplessly attacking repetition  \r\nOver, over left with who we are  \r\nDid you tell anyone?  \r\nDid you tell any place?  \r\n  \r\nEverybody else wanna fall in love  \r\nThere's no room for love in a modern sky  \r\nLiving in the era vulgaris  \r\nJ

In [38]:
# df = df.sample(5000)

In [39]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [40]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [41]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [42]:
nltk.download('punkt')
df['text'] = df['text'].apply(lambda x: tokenization(x))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\priya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [44]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [45]:
similarity[0]

array([1.        , 0.02890038, 0.00681733, ..., 0.01462113, 0.01157138,
       0.06063705])

In [46]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [47]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [49]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))